In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
from matplotlib import style
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact

import scipy.stats as stats
from scipy.stats import bernoulli
from scipy.stats import binom
from scipy.stats import beta
from scipy.stats import multivariate_normal

# import seaborn
import seaborn as sns
# settings for seaborn plotting style
sns.set(color_codes=True)
# settings for seaborn plot sizes
sns.set(rc={'figure.figsize':(5,5)})

<h1>Binary variables</h1>

<h2>Bernoulli distribution</h2>
<p>This is a distribution over a single binary variable $x \in \{0,1\}$ representing for example the result of flipping a coin. Its parameter is $\mu$ representing the probability that x=1. The conjugate prior for $\mu$ is the Beta distribution.</p>

$$Bern(x|\mu)=\mu^{x}(1-\mu)^{1-x}$$

$$E[x]=\mu $$

$$ var[x] = \mu(1-\mu) $$

In [2]:
@interact
def bernoulli_distribution(mu=(0.0,1.0)):
    fig, ax = plt.subplots(1, 1)
    x = np.arange(2)
    ax.plot(x, bernoulli.pmf(x,p=mu), 'bo', ms=8, label='bernoulli pmf')
    ax.vlines(x, 0, bernoulli.pmf(x,p=mu), colors='b', lw=5, alpha=0.5)
    ax.set(xlabel='Bernoulli Distribution', ylabel='Frequency')

interactive(children=(FloatSlider(value=0.5, description='mu', max=1.0), Output()), _dom_classes=('widget-inte…

<h2>Binomial distribution</h2>
<p>The binomial distribution gives the probability of observing m occurences of x=1 in a set of N samples where the probability of observing x=1 is $\mu \in [0,1]$. The conjugate prior for $\mu$ is the Beta distribution.</p>

$$
\begin{equation*}
Bin(m|N,\mu) = \frac{N!}{(N-m)!m!}\mu^{m}(1-\mu)^{N-m} 
\label{eq:binomial_dist} \tag{1}
\end{equation*}
$$

$$ E[m]=N\mu $$

$$ var[m]=N\mu(1-\mu) $$

In [3]:
@interact
def binomial_distribution(N=(1,100), mu=(0.0,1.0, 0.05)):
    fig, ax = plt.subplots(1, 1)
    x = np.arange(N+1)
    ax.xaxis.set_ticks(np.arange(0, N+2, int(round(N/11))))
    ax.plot(x, binom.pmf(x, N, mu), 'bo', ms=8, label='binom pmf')
    ax.vlines(x, 0, binom.pmf(x, N, mu), colors='b', lw=5, alpha=0.5)
    ax.set(xlabel='Binomial Distribution', ylabel='Frequency')

interactive(children=(IntSlider(value=50, description='N', min=1), FloatSlider(value=0.5, description='mu', ma…

<h1>Beta distribution</h1>

<p>The conjugate prior for the Bernoulli and Binomial distribution is the Beta distribution</p>

$$
\begin{equation*}
Beta(\mu|a,b) = \frac{\Gamma(a+b)}{\Gamma(a)\Gamma(b)}\mu^{a-1}(1-\mu)^{b-1} 
\label{eq:beta_dist} \tag{2}
\end{equation*}
$$

$$ 
\begin{equation*}
E[\mu] = \frac{a}{a+b} 
\label{eq:beta_expectation} \tag{3}
\end{equation*}
$$

$$ var[\mu] = \frac{ab}{(a+b)^2(a+b+1)} $$

In [4]:
@interact
def beta_distribution(a=(0.0,1000), b=(0.0,1000)):
    fig, ax = plt.subplots(1, 1)
    x = np.linspace(0,1,100)
    #ax.xaxis.set_ticks(np.arange(0, n, int(round(n/10))))
    ax.plot(x, beta.pdf(x, a, b), 'r-', lw=5, alpha=0.6, label='beta pdf')
    ax.set(xlabel='Beta Distribution')

interactive(children=(FloatSlider(value=500.0, description='a', max=1000.0), FloatSlider(value=500.0, descript…

<p>Let's suppose we have a data set $ D= {x_1, x_2, ... , x_N}$ where $x_i$ comes from a Bernoulli distribution and we want to estimate $\mu$. In a <b>frequentist</b> setting we can estimate $\mu$ by maximizing the <b>maximum likelihood</b>:</p>

$$ p(D|\mu) = \prod_{n=1}^N p(x_n|\mu) = \prod_{n=1}^N \mu^{x_n}(1-\mu)^{1-x_n}$$

or the log likelihood:

$$ \ln p(D|\mu) = \sum_{n=1}^N \ln p(x_n|\mu) = \sum_{n=1}^N [x_n\ln \mu + (1-x_n)\ln(1-\mu)]$$

This gives us

$$ \mu_{ML} = \frac{1}{N} \sum_{n=1}^N x_n = \frac{m}{N}$$ where m is the number of observations where x = 1. The above problem can be stated using the Binomial distribution.

<p>Consider our data set D has only one observation $x_1 = 1$. This will give us m=N=1 and $\mu_ML = 1$. This means that all future observations will be 1 which obviously is unreasonable.</p>

<p>To overcome this problem a <b>Bayesian</b> approachmight be helpful. For this we need a prior distribution $p(\mu)$. The conjugate prior for the Binomial distribution is the Beta distribution (2).</p>

<p>The posterior distribution of $\mu$ is obtained by multiplying the beta prior (2) with binomial likelihood function (1) and we get after normalization:</p>

$$ p(\mu|m,l,a,b)= \frac{\Gamma(m+a+l+b)}{\Gamma(m+a)\Gamma(l+b)}\mu^{m+a-1}(1-\mu)^{l+b-1} $$

We can see that a and b correspond to the number of observations of x=1 and x=0.

To predict the outcome of the next trial we compute the predictive <b>posterior predictive distribution</b> of x given the observed data D:

$$
\begin{aligned}
p(x=1|D) &= \int_{0}^1 p(x=1,\mu|D)d\mu \\
         &= \int_{0}^1 p(x=1|\mu,D)p(\mu|D)d\mu \\
         &= \int_{0}^1 \mu p(\mu|D) \\
         &= E[\mu|D] \\
         &= \frac{m+a}{m+a+l+b}
\end{aligned}
$$

<h1>Multinomial variables</h1>

<h2>Multinomial distribution</h2>
Represents variables that can take one of K states. Ex:

$$ x=(0,0,1,0,0,0)^T $$ here K=6 and $x_3=1$. Wesay $\mu_k$ is the probability that $x_k=1$. Hence the distribution of x is given by:

$$ p(x|\mu)= \prod_{k=1}^K \mu_{k}^{x_k} $$ where 

$$ \mu = (\mu_1... \mu_K)^T $$

We have 

$$ E[x|\mu] = \mu $$

<p>Let's denote D a set of N independent observations ${x_1 ... x_N}$. Then the likelihood function is:</p>

$$ 
\begin{aligned}
p(D|\mu) &= \prod_{n=1}^N \prod_{k=1}^K \mu_{k}^{x_{nk}} \\
         &= \prod_{k=1}^K \mu_{k}^{(\sum_nx_{nk})} \\
         &= \prod_{k=1}^K \mu_k^{m_k}
\end{aligned}
$$

where $m_k = \sum_n x_{nk}$ which is the number of observations of $x_k=1$

We find the maximum likelihood for $\mu$ by maximizing $\ln p(D|\mu)$ and get:

$$ \mu_k^{ML}=\frac{m_k}{N} $$

The <b>multinomial</b> distribution is :

$$ 
\begin{equation*}
Mult(m_1, m_2, ..., m_K|\mu, N) = \frac{N!}{m_1!m_2!...m_K!}\prod_{k=1}^K \mu_k^{m_k} 
\label{eq:mult_dist} \tag{4}
\end{equation*}$$

<h2>Dirichlet distribution</h2>
<p>The conjugate prior for the multinomial distribution is Dirichlet distribution:</p>

$$ 
\begin{equation*}
Dir(\mu|\alpha) = \frac{\Gamma(\alpha_1 +...+ \alpha_K)}{\Gamma(\alpha_1)...\Gamma(\alpha_K)}\prod_{k=1}^K \mu_k^{\alpha_k-1}
\label{eq:dir_dist} \tag{5}
\end{equation*}
$$

We can get the posterior distribution for $\mu$ by multiplying the likelihood function (4) with the prior (5). After normalization we get:

$$ 
\begin{aligned}
p(\mu|D,\alpha) &= Dir(\mu|\alpha+m) \\
                &= \frac{\Gamma(\alpha_1 +...+ \alpha_K + N)}{\Gamma(\alpha_1+m_1)...\Gamma(\alpha_K+m_K)}\prod_{k=1}^K \mu_k^{\alpha_k+m_k-1}
\end{aligned}
$$ where $\alpha_k $ is the number of observations of $x_k=1$.

<h1>Continuous variables</h1>

<h2>Gaussian distribution</h2>

$$ N(x|\mu, \Sigma) = \frac{1}{(2\pi^{\frac{D}{2}})} \frac{1}{|\Sigma|^\frac{1}{2}} e^{-\frac{1}{2}(x-\mu)^T\Sigma^{-1}(x-\mu)}$$

where $\mu$ is a D dimensioanl vector and $\Sigma$ is a DxD covariance matrix given by $ \Sigma_{i,j} = \sigma(x_i,y_j) $

$$ E[x]=\mu $$

$$ cov[x]=\Sigma $$

In [5]:
@interact
def gaussian_distribution(cov_xx=(1,20,0.5), cov_yy=(1,20,0.5)):
    mu_x = 0
    variance_x = 3

    mu_y = 0
    variance_y = 15

    x = np.linspace(-10,10,500)
    y = np.linspace(-10,10,500)
    X,Y = np.meshgrid(x,y)

    pos = np.array([X.flatten(),Y.flatten()]).T

    rv = multivariate_normal([mu_x, mu_y], [[cov_xx, 0], [0, cov_yy]])

    fig = plt.figure(figsize=(10,10))
    ax0 = fig.add_subplot(111)
    ax0.contour(rv.pdf(pos).reshape(500,500))

    plt.show()
    

interactive(children=(FloatSlider(value=10.0, description='cov_xx', max=20.0, min=1.0, step=0.5), FloatSlider(…

<h3>Partitioned Gaussians</h3>

<img src="images/1.png">

Given a joint Gaussian distribution $N(x|\mu, \Sigma)$ with $\Lambda \equiv \Sigma^{-1}$ and 
$
x= \begin{pmatrix}
    x_a \\
    x_b 
    \end{pmatrix}
$ ,
$
\mu= \begin{pmatrix}
     \mu_a \\
     \mu_b 
     \end{pmatrix}
$,
$
\Sigma = \begin{pmatrix}
         \Sigma_{aa} & \Sigma_{ab} \\
         \Sigma_{ba} & \Sigma_{bb}
         \end{pmatrix}
$,
$
\Lambda = \begin{pmatrix}
          \Lambda{aa} & \Lambda{ab} \\
          \Lambda{ba} & \Lambda{bb}
          \end{pmatrix}
$

Conditional distribution:

$$ p(x_a|x_b) = N(x|\mu_{a|b}, \Lambda_{aa}^{-1}) $$
$$ \mu_{a|b} = \mu_a - \Lambda^{-1}_{aa} \Lambda_{ab}(x_b-\mu_b) $$

Marginal distribution:

$$ p(x_a) = N(x_a|\mu_a, \Sigma_{aa}) $$

<h3>Bayes' theorem for Gaussian variables</h3>

We are given a Gaussian marginal distribution p(x) and a Gaussian conditional distribution p(y|x) which has a mean that is a linear function of x anf covariance independent of x(as above). We want to find the marginal distribution p(y)and conditional distribution p(y|x). We have:

$$ p(x) = N(x|\mu,\Lambda^{-1}) $$
$$ p(y|x) = N(y|Ax+b, L^{-1}) $$

where $\mu$, $A$ and $b$ are parameters governing the means and $\Lambda$ and $L$ are precisions matrices.

we get the following <b>marginal</b> and <b>conditional Gausian</b>:

$$ p(y) = N(y|A\mu+b, L^{-1}+A\Lambda^{-1}A^T) $$

$$ p(x|y) = N(x|\Sigma{A^TL(y-b) + \Lambda\mu}, \Sigma) $$

where $$\Sigma = (\Lambda + A^TLA)^{-1}$$

<h3>Maximum likelihood for the Gaussians</h3>

Given a data set $ X={x_1, ... x_N} $ where each observation is drawn independently from a multivariate Gaussian we can estimate the parameters of the distribution by ML. The derivative of the log likelihood with respect to $ \mu $ is :

$$ \frac{\partial}{\partial\mu} \ln p(X|\mu,\Sigma) = \sum_{n=1}^{N}\Sigma^{-1}(x_n-\mu) $$

Setting this to 0 gives:

$$ \mu_{ML} = \frac{1}{N}\sum_{n=1}^{N}x_n $$

We can get the maximization for $\Sigma$ and get:

$$ \Sigma_{ML} = \frac{1}{N}\sum_{n=1}^{N}(x_n-\mu_{ML})(x_n-\mu_{ML})^T $$

<h3>Bayesian inference for the Gaussian</h3>

The bayesian approsch involves prior distributions.
<p></p>
<b>Case I - $\sigma$ is known and infer $\mu$</b>
We have N observations $X={x_1, ... x_N}$, then the likelihood function is:

$$ p(x|\mu) = \prod_{n=1}^N p(x_n|\mu) = \frac{1}{(2\pi\sigma^2)^{N/2}}e^{{-\frac{1}{2\sigma^2}\sum_{n=1}^{N}(x_n-\mu)^2}} $$

The conjugate prior for this function is a Gaussian so we take the prior function to be:

$$ p(\mu) = N(\mu|\mu_0, \sigma_0^2) $$

The posterior distribution then becomes

$$ p(\mu|x) \propto p(x|\mu)p(\mu) $$

$$ p(\mu|x) = N(\mu|\mu_N,\sigma_N^2) $$

where

$$ \mu_N = \frac{\sigma^2}{N\sigma_0^2 + \sigma^2}\mu_0 + \frac{N\sigma_0^2}{N\sigma_0^2 + \sigma^2}\mu_{ML} $$

$$ \frac{1}{\sigma_N^2} = \frac{1}{\sigma_0^2 + \frac{N}{\sigma^2}} $$

$$ \mu_{ML} = \frac{1}{N}\sum_{n=1}^{N}x_n $$

In [6]:
# Definition of the prior distribution p(\mu)
@interact
def prior_gaussian(mu=(-2,2,0.1), variance=(0.1,0.1)):
    sigma = math.sqrt(variance)
    x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
    plt.xlim([-1, 1])
    plt.ylim([0, 5])
    plt.plot(x, stats.norm.pdf(x, mu, sigma))
    plt.show()

interactive(children=(FloatSlider(value=0.0, description='mu', max=2.0, min=-2.0), FloatSlider(value=0.1, desc…

In [7]:
# Generetae new data from a Gaussian witm mean 0.8 and variance 0.1
def generate_gaussian(N):
    mu, sigma = 0.8, 0.1
    samples = np.random.normal(mu, sigma, N)
    return samples

In [8]:
#Compute the posterior distribution p(\mu, x), after observing N data examples
@interact
def posterior_gaussian(N=[1,2,10]):
    var = 0.1
    var0 = 0.1
    sig = math.sqrt(var)
    sig0 = math.sqrt(var0)
    mu0=0
    muML= np.mean(generate_gaussian(N))
    muN = (var/(N*var0 + var))*mu0 + (N*var0)/(N*var0 + var)*muML
    sigmaN = pow((1/var0 + N/var),-1)
    print(sigmaN)
    x = np.linspace(-5, 5, 1000)
    plt.xlim([-1, 1])
    plt.ylim([0, 5])
    plt.plot(x, stats.norm.pdf(x, muN, math.sqrt(sigmaN)))
    plt.show()

interactive(children=(Dropdown(description='N', options=(1, 2, 10), value=1), Output()), _dom_classes=('widget…

<h3>Mixtures of Gaussians</h3>

<img src="images/2.png">

Linear combinations of basic distributions such as Gaussians are called <b>mixture distributions</b>. A <b>mixture of Gaussians</b> with K components has the form:

$$ p(x) = \sum_{k=1}^K \pi_k N(x|\mu_k, \Sigma_k)$$

Parameters $\pi_k$ are called mixing coefficients and 

$$ \sum_{k=1}^K \pi_k = 1 $$

<img src="images/3.png">

The mixing coefficients satisfy the requirements to be probabilities. The marginal density can be stated as:

$$ p(x) = \sum_{k=1}^K p(k)p(x|k) $$ which is exactly the formula above if we consider $ \pi_k = p(k) $ and $ N(x|\mu_k, \Sigma_k) = p(x|k) $

The posterior probabilities $ p(k|x) $ are known as responsabilities and are given by:

$$ 
\begin{aligned}
\gamma_k(x) &= p(k|x) \\
            &= \frac{p(k)p(x|k)}{\sum_l p(l)p(x|l)} \\
            &= \frac{\pi_k N(x|\mu_k, \Sigma_k)}{\sum_l \pi_l N(x|\mu_l, \sum_l)}
\end{aligned}
$$

<h2>The Exponential family</h2>

The exponential family of distributions over x, given parameters $\eta$ is defined as:

$$ p(x|\eta) = h(x)g(\eta)exp\{\eta^Tu(x)\} $$

where $\eta$ is called the natural parameter, u(x) is a function of x and g($\eta$) ensures the distribution is normalized.

Let's consider the <b>Bernoulli distribution</b>:

$$ 
\begin{aligned}
p(x|\mu) &= Bern(x|\mu) \\
         &= \mu^x(1-\mu)^{1-x} \\
         &= e^{x\ln\mu + (1-x)\ln(1-\mu)} \\
         &= (1-\mu)e^{\ln(\frac{\mu}{1-\mu})x}
\end{aligned}
$$

This, we can identify:

$$ \eta = \ln(\frac{\mu}{1-\mu}) $$ and having $\mu = \sigma(\eta)$ we get

$$ \sigma(\eta) = \frac{1}{1+e^{-\eta}} $$

<h2>Nonparametric methods</h2>

The <b>parametric</b> approach tries to determine the parameters of the probability distributions using a data set. The <b>nonparametric</b> approach makes few assumtions about the form of distribution.
    
<p></p>
The histograms methods for density estimation simply partition x into dinstinct bins of width $\Delta_i$ and then count the number $n_i$ of observations of x falling in bin i. To turn it into a normalized probability density we get:

$$ p_i = \frac{n_i}{N\Delta_i} $$

<img src="images/4.png">

<h3>Kernel density estimators</h3>

Let us suppose that observations are being drawn from some unknown probability density p(x) in some D-dimensional space and we want to estmate p(x). Let us consider some small region R containing x. The probability mass associated with this region is given by:

$$ P = \int_{R}p(x)dx $$

Now suppose that we have collected a data set comprising N observations drawn from p(x). Because each data point has a probability P of falling within R, the total number K of points that lie inside R will be distributed according to the binomial distribution:

$$ Bin(K|N,P) = \frac{N!}{K!(N-K)!}P^K(1-P)^{1-K} $$

We have $E[K/N] = P$ and for large N

$$ K \approx NP $$

We also assume that the region R is sufficiently small that the probability density p(x) is roughly constant over the region, then we have:

$$ P \approx p(x)V $$

Combining we get:

$$ p(x) = \frac{K}{NV} $$

If we fix K and determine V, we have K-nerest-neighboor, if we fix V and determine K we get the kernel approach. In this approach we takethe region R to be a small hypercube centred on the point x at which we wish to determine the probability density. In order to count the number K of points falling within this region, it is convenient to define the following function:

$$ 
k(u) = \left\{
    \begin{array}\\
        1, & |u_i| <= 1/2, i=1...D \\
        0 & otherwise
    \end{array}
\right.
$$

which represents a unit cube centred on the origin. The function k(u) is an example of a kernel function. The quantity $ k((x-x_n)/h) $ will be one if the data point $x_n$ lies inside a cube of side h centred on x, and zero otherwise. The total number of data points lying inside this cube will therefore be: 

$$ K = \sum_{n=1}^N k(\frac{x-x_n}{h}) $$

Substituing we get:

$$ p(x) = \frac{1}{N}\sum_{n=1}^N \frac{1}{h^D}k(\frac{x-x_n}{h}) $$
where $V=h^D$

We can interpret this as the sum over N cubes centred on the N data points $x_n$.

We can obtain a smoother density model if we choose a smoother kernel function, and a common choice is the Gaussian

$$ p(x) = \frac{1}{N} \sum_{n=1}^N \frac{1}{(2\pi h^2)^{1/2}} e^{-\frac{||x-x_n||^2}{2h^2}} $$

where h represents the standard deviation of the Gaussian components.

<img src="images/5.png">

<h3>Nearest-neighbour methods</h3>

We consider a fixed value of K and use the data to find an appropriate value for V. To do this, we consider a small sphere centred on the point x at which we wish to estimate the density p(x), and we allow the radius of the sphere to grow until it contains precisely K data points.

<img src="images/6.png">